### Min Cost Objective

$$ \min \sum_{t=1}^{7}Labor\_Cost\_Doctor * D_t + Labor\_Cost\_Nurse * N_t + C\_N\_Mask * N\_Mask_t + C\_S\_Mask * S\_Mask_t + C\_Vent * Vent_t + C\_Glove * Glove_t + C\_Gown * Gown_t + C\_Eye\_Pro * Eye\_Pro_t + C\_Flu * Flu_t + C\_Staff\_Gown * Staff\_Gown_t$$
   s.t.  
   $$ 4\_D\_Req * D_t + 4\_N\_Req * N_t \ge 4\_Pat_t$$
   $$ 3\_D\_Req * D_t + 3\_N\_Req * N_t \ge 3\_Pat_t$$
   $$ 2\_D\_Req * D_t + 2\_N\_Req * N_t \ge 2\_Pat_t$$
   $$ 1\_N\_Req * N_t \ge 1\_Pat_t $$
   $$ D_t = D_{t-1} + On\_D_t - Off\_D_t $$
   $$ N_t = N_{t-1} + On\_N_t - Off\_N_t $$
   $$Adm\_Pat_t = 4\_Pat_t + 3\_Pat_t + 2\_Pat_t $$
   $$Staff_t = D_t + N_t $$
   $$Tot\_Pat_t = 4\_Pat_t + 3\_Pat_t + 2\_Pat_t + 1\_Pat_t $$
   $$ Adm\_Pat_t \le Num\_Beds $$
   $$ Gown_{t-1} + Gown\_Order_t = Adm\_Pat_t + Gown_t $$
   $$ Eye\_Pro_{t-1} + Eye\_Pro\_Order_t = Staff_t + Eye\_Pro_t $$
   $$ N\_Mask_{t-1} + N\_Mask\_Order_t = Staff_t + N\_Mask_t $$
   $$ Staff\_Gown_{t-1} + Staff\_Gown\_Order_t = Staff_t + Staff\_Gown_t $$
   $$ S\_Mask_{t-1} + S\_Mask\_Order_t = Tot\_Pat_t + S\_Mask_t $$
   $$ Flu_{t-1} + Flu\_Order_t = Req\_Flu * Adm\_Pat_t + Flu_t $$
   $$ Glove_{t-1} + Glove\_Order_t = Req\_Glove * Staff_t + Glove_t$$
   $$ Vent_{t-1} + Vent\_Order_t = 4\_Pat_t + 3\_Pat_t + Vent_t $$
   $$ Gown_0 = Int\_Gown $$
   $$ Eye\_Pro_0 = Int\_Eye\_Pro $$
   $$ N\_Mask_0 = Int\_N\_Mask $$
   $$ Staff\_Gown_0 = Int\_Staff\_Gown $$
   $$ S\_Mask_0 = Int\_S\_Mask $$
   $$ Flu_0 = Int\_Flu $$
   $$ Glove_0 = Int\_Glove $$
   $$ Gown_t, Vent_t, Eye\_Pro_t, N\_Mask_t, Staff\_Gown_t, S\_Mask_t, Flu_t, Glove_t, D_t, N_t, Gown\_Order_t, Eye\_Pro\_Order_t, N\_Mask\_Order_t, Staff\_Gown\_Order_t, S\_Mask\_Order_t, Flu\_Order_t, Glove\_Order_t, Vent\_Order_t \ge 0 $$

### Patient Treatment Objective

* Create Treatment Path  

* Include time for each step and capacities for each step  

* Create logical constraints for different patient severities


In [8]:
using Random, StatsBase, NamedArrays


# Creating an array of new cases each day based on the statisical data
new_cases = [rand(6:16) for i in 1:1, j in 1:7] 

# Finding the number of Covid cases that required hospitalization from
# CDC data 
hospitalized = round.(Int, new_cases*0.1642) 

# Patients with severity 1 are those who don't need hospitalization
pat_1 = new_cases - hospitalized 

# Initialize categories of hospitalized patients and probabilities
items = ["2_Pat", "3_Pat", "4_Pat"]
weights = [0.6232, 0.206, 0.1707]

# Distribute new cases among our categories
distr = [sample(items, Weights(weights), i) for i in hospitalized]

# Distribute current cases among our categories
prior_pats = sample(items, Weights(weights), 41);
c = countmap(prior_pats)

# Initialize patient category arrays
pat_2 = zeros(1,7)
pat_3 = zeros(1,7)
pat_4 = zeros(1,7)

# Separate patients from new hospitalizations into category arrays
for i in 1:length(distr)
    for j in distr[i]
        if j == "4_Pat"
            pat_4[i] += 1
        end
        if j == "3_Pat"
            pat_3[i] += 1
        end
        if j == "2_Pat"
            pat_2[i] += 1
        end
    end
end

# Compute total daily patients for severity 3
count_3 = c["3_Pat"]
for i in 1:length(pat_3)
    if pat_3[i] == 0
        pat_3[i] = count_3
    else
        count_3 = count_3 + pat_3[i] 
        pat_3[i] = count_3
    end
end

# Compute total daily patients for severity 4
count_4 = c["4_Pat"]
for i in 1:length(pat_4)
    if pat_4[i] == 0
        pat_4[i] = count_4
    else
        count_4 = count_4 + pat_4[i] 
        pat_4[i] = count_4
    end
end

# Compute total daily patients for severity 2
count_2 = c["2_Pat"]
for i in 1:length(pat_2)
    if pat_2[i] == 0
        pat_2[i] = count_2
    else
        count_2 = count_2 + pat_2[i] 
        pat_2[i] = count_2
    end
end

# Combine categories into full demand schedule
days = [1, 2, 3, 4, 5, 6, 7]
sev = [1, 2, 3, 4]
Patients = [pat_1; pat_2; pat_3; pat_4]
demand = NamedArray(Patients, (sev, days), ("Severity Level", "Day"))

4×7 Named Array{Float64,2}
Severity Level ╲ Day │    1     2     3     4     5     6     7
─────────────────────┼─────────────────────────────────────────
1                    │ 11.0   8.0  11.0  10.0  10.0  12.0   8.0
2                    │ 25.0  27.0  29.0  29.0  30.0  30.0  31.0
3                    │ 10.0  10.0  10.0  12.0  12.0  14.0  14.0
4                    │  8.0   8.0   8.0   8.0   9.0   9.0   9.0

In [20]:
using JuMP, Clp

# supply costs
d_cost = 393.69
n_cost = 180.47
n95_mask_cost = 6.00
surg_mask_cost = 0.50
gown_cost = 4.15
glove_cost = 0.25
shield_cost = 5.00
vent_cost = 25000.00
fluid_cost = 5.00
staff_gown_cost = 2.00


# initial supplies
gown_init = 1320
staff_gown_init = 1320
glove_init = 8250
n95_mask_init = 186
surg_mask_init = 330
shield_init = 69
vent_init = 16
fluid_init = 71904
d_init = 80
n_init = 160

# constants
num_beds = 102 # total in hospital
req_staff_gown = 440 # per staff per day

# supplies needed per patient per day
req_gown = 20
req_staff_gown = 20
req_glove = 250
req_n95_mask = 6
req_surg_mask = 10
req_fluid = 2179

# staff needed per patient per day depending on patient severity
n_req_1 = 0.25
n_req_2 = 0.25
n_req_3 = 0.5
n_req_4 = 0.5
d_req_2 = 0.05
d_req_3 = 0.0714286
d_req_4 = 0.0714286

m = Model(Clp.Optimizer)

# gowns
@variable(m, gown[1:7] >= 0)
@variable(m, gown_order[1:7] >= 0)
@variable(m, staff_gown[1:7] >= 0)
@variable(m, staff_gown_order[1:7] >= 0)

# gloves
@variable(m, glove[1:7] >= 0)
@variable(m, glove_order[1:7] >= 0)

# masks
@variable(m, n95_mask[1:7] >= 0)
@variable(m, n95_mask_order[1:7] >= 0)
@variable(m, surg_mask[1:7] >= 0)
@variable(m, surg_mask_order[1:7] >= 0)

# eye protection
@variable(m, shield[1:7] >= 0)
@variable(m, shield_order[1:7] >= 0)

# ventilators
@variable(m, vent[1:7] >= 0)
@variable(m, vent_order[1:7] >= 0)

# flu patients
@variable(m, fluid[1:7] >= 0)
@variable(m, fluid_order[1:7] >= 0)

# doctors and nurses on shift
@variable(m, n[1:7] >= 0)
@variable(m, d[1:7] >= 0)

# doctors and nurses added to shift
@variable(m, n_on[1:7] >= 0)
@variable(m, d_on[1:7] >= 0)

# doctors and nurses taken off of shift
@variable(m, n_off[1:7] >= 0)
@variable(m, d_off[1:7] >= 0)

# patients
# @constraint(m, bed_constr[days in 1:7], (pat_2[days] + pat_3[days] + pat_4[days]) <= num_beds)

# staff
@constraint(m, nurse_bal_init, n[1] == n_init + n_on[1] - n_off[1])
@constraint(m, nurse_bal[days in 2:7], n[days] == n[days-1] + n_on[days] - n_off[days])

@constraint(m, doctor_bal_init, d[1] == d_init + d_on[1] - d_off[1])
@constraint(m, doctor_bal[days in 2:7], d[days] == d[days-1] + d_on[days] - d_off[days])


@constraint(m, sev_1_req[days in 1:7], pat_1[days] <= n_req_1 * n[days])
@constraint(m, sev_2_req[days in 1:7], pat_2[days] <= n_req_2 * n[days] + d_req_2 * d[days])
@constraint(m, sev_3_req[days in 1:7], pat_3[days] <= n_req_3 * n[days] + d_req_3 * d[days])
@constraint(m, sev_4_req[days in 1:7], pat_4[days] <= n_req_4 * n[days] + d_req_4 * d[days])

# supplies
@constraint(m, gown_inv_bal_init, gown_init + gown_order[1] == req_gown * (pat_2[1] + pat_3[1] + pat_4[1]) + gown[1])
@constraint(m, gown_inv_bal[days in 2:7], gown[days-1] + gown_order[days] == req_gown * (pat_2[days] + pat_3[days] + pat_4[days]) + gown[days])

@constraint(m, staff_gown_inv_bal_init, staff_gown_init + staff_gown_order[1] == req_staff_gown * (pat_1[1] + pat_2[1] + pat_3[1] + pat_4[1]) + staff_gown[1])
@constraint(m, staff_gown_inv_bal[days in 2:7], staff_gown[days-1] + staff_gown_order[days] == req_staff_gown * (pat_1[days] + pat_2[days] + pat_3[days] + pat_4[days]) + staff_gown[days])

@constraint(m, glove_inv_bal_init, glove_init + glove_order[1] == req_glove * (pat_1[1] + pat_2[1] + pat_3[1] + pat_4[1]) + glove[1])
@constraint(m, glove_inv_bal[days in 2:7], glove[days-1] + glove_order[days] == req_glove * (pat_1[days] + pat_2[days] + pat_3[days] + pat_4[days]) + glove[days])

@constraint(m, n95_mask_inv_bal_init, n95_mask_init + n95_mask_order[1] == req_n95_mask * (pat_1[1] + pat_2[1] + pat_3[1] + pat_4[1]) + n95_mask[1])
@constraint(m, n95_mask_inv_bal[days in 2:7], n95_mask[days-1] + n95_mask_order[days] == req_n95_mask * (pat_1[days] + pat_2[days] + pat_3[days] + pat_4[days]) + n95_mask[days])

@constraint(m, surg_mask_inv_bal_init, surg_mask_init + surg_mask_order[1] == req_surg_mask * (pat_1[1] + pat_2[1] + pat_3[1] + pat_4[1]) + surg_mask[1])
@constraint(m, surg_mask_inv_bal[days in 2:7], surg_mask[days-1] + surg_mask_order[days] == req_surg_mask * (pat_1[days] + pat_2[days] + pat_3[days] + pat_4[days]) + surg_mask[days])

@constraint(m, shield_inv_bal_init, shield_init + shield_order[1] == (d[1] + n[1]) + shield[1])
@constraint(m, shield_inv_bal[days in 2:7], shield[days-1] + shield_order[days] == (d[days] + n[days]) + shield[days])

@constraint(m, vent_inv_bal_init, vent_init + vent_order[1] == (pat_4[1] + pat_3[1]) + vent[1])
@constraint(m, vent_inv_bal[days in 2:7], vent[days-1] + vent_order[days] == (pat_4[days] + pat_3[days]) + vent[days])

@constraint(m, fluid_inv_bal_init, fluid_init + fluid_order[1] == req_fluid * (pat_2[1] + pat_3[1] + pat_4[1]) + fluid[1])
@constraint(m, fluid_inv_bal[days in 2:7], fluid[days-1] + fluid_order[days] == req_fluid * (pat_2[days] + pat_3[days] + pat_4[days]) + fluid[days])

@objective(m, Min, d_cost * sum(d) + n_cost * sum(n) + n95_mask_cost * sum(n95_mask) + surg_mask_cost * sum(surg_mask) + gown_cost * sum(gown) + glove_cost * sum(glove) + shield_cost * sum(shield) + vent_cost * sum(vent) + fluid_cost * sum(fluid) + staff_gown_cost * sum(staff_gown))


393.69 d[1] + 393.69 d[2] + 393.69 d[3] + 393.69 d[4] + 393.69 d[5] + 393.69 d[6] + 393.69 d[7] + 180.47 n[1] + 180.47 n[2] + 180.47 n[3] + 180.47 n[4] + 180.47 n[5] + 180.47 n[6] + 180.47 n[7] + 6 n95_mask[1] + 6 n95_mask[2] + 6 n95_mask[3] + 6 n95_mask[4] + 6 n95_mask[5] + 6 n95_mask[6] + 6 n95_mask[7] + 0.5 surg_mask[1] + 0.5 surg_mask[2] + 0.5 surg_mask[3] + 0.5 surg_mask[4] + 0.5 surg_mask[5] + 0.5 surg_mask[6] + 0.5 surg_mask[7] + 4.15 gown[1] + 4.15 gown[2] + 4.15 gown[3] + 4.15 gown[4] + 4.15 gown[5] + 4.15 gown[6] + 4.15 gown[7] + 0.25 glove[1] + 0.25 glove[2] + 0.25 glove[3] + 0.25 glove[4] + 0.25 glove[5] + 0.25 glove[6] + 0.25 glove[7] + 5 shield[1] + 5 shield[2] + 5 shield[3] + 5 shield[4] + 5 shield[5] + 5 shield[6] + 5 shield[7] + 25000 vent[1] + 25000 vent[2] + 25000 vent[3] + 25000 vent[4] + 25000 vent[5] + 25000 vent[6] + 25000 vent[7] + 5 fluid[1] + 5 fluid[2] + 5 fluid[3] + 5 fluid[4] + 5 fluid[5] + 5 fluid[6] + 5 fluid[7] + 2 staff_gown[1] + 2 staff_gown[2] + 2 sta